In [ ]:
%matplotlib inline
%pylab inline

import menpo.io as mio
from menpo.feature import fast_dsift, no_op
from menpo.landmark import labeller, ibug_face_66
from menpo.visualize import visualize_images

from menpofit.lk import (
    LKFitter,
    FA, FC, IC,
    SSD, FourierSSD, ECC, GradientImages, GradientCorrelation)
from menpofit.transform import DifferentiableAlignmentAffine
from menpofit.visualize import visualize_aam

In [2]:
import IPython.display as ipydisplay
from menpo.visualize.viewmatplotlib import sample_colours_from_colourmap
from menpofit.fittingresult import plot_cumulative_error_distribution

## Load data

In [ ]:
images = []
for i in mio.import_images('/vol/atlas/databases/lfpw/trainset/', 
                           verbose=True, max_images=5):
    labeller(i, 'PTS', ibug_face_66)
    i.crop_to_landmarks_proportion_inplace(0.5, group='PTS')
    i.landmarks['bounding_box'] = i.landmarks['PTS'].lms.bounding_box()
    if i.n_channels == 3:
        i = i.as_greyscale(mode='average')
    images.append(i)

In [ ]:
template = images[0]
test_images = images

In [ ]:
visualize_images(template)

In [ ]:
visualize_images(test_images)

## Build LK Fitter

In [ ]:
lk_fitter = LKFitter(
    template=template,
    group='bounding_box',
    features=no_op,
    transform_cls=DifferentiableAlignmentAffine,
    diagonal=100, 
    scales=(1, 0.5), 
    scale_features=True,
    algorithm_cls=IC,
    residual_cls=SSD)

## Fit LK Fitter

In [ ]:
np.random.seed(seed=1)

fitter_results = []

for j, i in enumerate(test_images):
    
    gt_s = i.landmarks['bounding_box'].lms
    s = lk_fitter.perturb_shape(gt_s, noise_std=10)
    
    fr = lk_fitter.fit(i, s, gt_shape=gt_s, max_iters=20) 
    fitter_results.append(fr)
    
    print 'Image: ', j
    print fr

## Visualize Results

In [ ]:
fitted_images = [fr.fitted_image for fr in fitter_results]

In [ ]:
visualize_images(fitted_images)